In [ ]:
!pip install ultralytics
!pip install -q gradio tensorflow pillow numpy


In [ ]:
# ========================================
# GOOGLE COLAB SETUP - Run this first!
# ========================================

# Install required packages
# !pip install -q gradio tensorflow pillow

print("✅ Setup complete!")

# ========================================
# MAIN CODE
# ========================================

import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image

# ------------------------
# LOAD MODELS
# ------------------------

print("Loading models...")

# Stage 1: Species classifier (Cow vs Buffalo)
try:
    species_interpreter = tf.lite.Interpreter(model_path="classifier.tflite")
    species_interpreter.allocate_tensors()
    species_input = species_interpreter.get_input_details()[0]
    species_output = species_interpreter.get_output_details()[0]
    SPEC_IMG_SIZE = int(species_input["shape"][1])
    print(f"✓ Species classifier loaded (input size: {SPEC_IMG_SIZE})")
except Exception as e:
    print(f"❌ Error loading species classifier: {e}")
    raise

# Stage 2a: Cow breed classifier
try:
    cow_interpreter = tf.lite.Interpreter(model_path="indian_breed_model_cow.tflite")
    cow_interpreter.allocate_tensors()
    cow_input = cow_interpreter.get_input_details()[0]
    COW_IMG_SIZE = int(cow_input["shape"][1])
    print(f"✓ Cow breed classifier loaded (input size: {COW_IMG_SIZE})")
except Exception as e:
    print(f"❌ Error loading cow classifier: {e}")
    raise

# Stage 2b: Buffalo breed classifier
try:
    buff_interpreter = tf.lite.Interpreter(model_path="indian_breed_model_bafflow.tflite")
    buff_interpreter.allocate_tensors()
    buff_input = buff_interpreter.get_input_details()[0]
    BUFF_IMG_SIZE = int(buff_input["shape"][1])
    print(f"✓ Buffalo breed classifier loaded (input size: {BUFF_IMG_SIZE})")
except Exception as e:
    print(f"❌ Error loading buffalo classifier: {e}")
    raise

print("\n🎉 All models loaded successfully!\n")

# ------------------------
# LABELS
# ------------------------

# ✅ CORRECTED: Model outputs single sigmoid value
# Output > 0.5 = Cow (label 1)
# Output < 0.5 = Buffalo (label 0)
species_labels = ["Cow", "Buffalo"]

cow_labels = [
    'Alambadi', 'Gir', 'Hariana', 'Holstein_Friesian', 'Kangayam', 'Kankrej',
    'Khillari', 'Rathi', 'Red kandhari', 'Red_Sindhi', 'Sahiwal', 'Tharparkar',
    'bachaur', 'bhelai', 'gangatari', 'gaolao', 'kherigarh'
]

buff_labels = [
    'Jaffrabadi', 'Mehsana', 'Murrah', 'Neli Ravi', 'banni', 'pandharpuri'
]

# ------------------------
# HELPER FUNCTIONS
# ------------------------

def run_tflite(interpreter, tensor):
    """Run inference on TFLite model"""
    inp = interpreter.get_input_details()[0]["index"]
    out = interpreter.get_output_details()[0]["index"]
    interpreter.set_tensor(inp, tensor)
    interpreter.invoke()
    return interpreter.get_tensor(out)[0]

def entropy(prob):
    """Calculate entropy for confidence assessment"""
    return -np.sum(prob * np.log(prob + 1e-9))

def purity(ent, top1, top2):
    """Determine if breed is pure or crossbreed"""
    if ent > 1.8 and abs(top1 - top2) < 0.15:
        return "❗ Likely Crossbreed"
    if top1 < 0.60 and abs(top1 - top2) < 0.10:
        return "❗ Crossbreed (Low Confidence)"
    if ent > 2.2:
        return "⚠ Unknown Breed"
    return "✅ Pure Breed"

# ------------------------
# MAIN PIPELINE
# ------------------------

def run_species_and_breed(image_pil):
    """
    Two-stage pipeline:
    Stage 1: Species classification (Cow vs Buffalo)
    Stage 2: Breed classification (based on species)
    """
    try:
        # Stage 1: Species Classification
        spec_pil = image_pil.resize((SPEC_IMG_SIZE, SPEC_IMG_SIZE))
        spec_tensor = np.expand_dims(np.array(spec_pil, dtype=np.float32), 0)
        spec_pred = run_tflite(species_interpreter, spec_tensor)

        # Handle both binary classification outputs:
        # Case 1: Single output (sigmoid) - outputs buffalo probability
        # Case 2: Two outputs (softmax) - outputs [buffalo_prob, cow_prob]

        if len(spec_pred) == 1:
            # Single output: sigmoid activation
            # Output value represents COW probability (closer to 1 = cow, closer to 0 = buffalo)
            cow_prob = float(spec_pred[0])
            buffalo_prob = 1.0 - cow_prob
            spec_probs = [cow_prob, buffalo_prob]

            # Determine species based on threshold
            if cow_prob > 0.5:
                spec_idx = 0  # Cow
                spec_conf = cow_prob
            else:
                spec_idx = 1  # Buffalo
                spec_conf = buffalo_prob
        else:
            # Two outputs: softmax activation
            spec_probs = spec_pred
            spec_idx = int(np.argmax(spec_pred))
            spec_conf = float(spec_pred[spec_idx])

        species_name = species_labels[spec_idx] if spec_idx < len(species_labels) else f"Unknown_{spec_idx}"

        # 🔍 DEBUG: Show all probabilities to verify label order
        debug_info = "\n**🔍 Debug - Species Probabilities:**\n"
        for i, label in enumerate(species_labels):
            debug_info += f"- {label}: {spec_probs[i]*100:.2f}%\n"

        # ⚠️ CONFIDENCE CHECK: Ensure species prediction is reliable
        if spec_conf < 0.60:
            txt = f"### ⚠️ Low Species Confidence\n\n"
            txt += f"**Predicted:** {species_name} ({spec_conf*100:.2f}%)\n\n"
            txt += debug_info
            txt += "\n**Recommendation:** The image may be unclear or the animal is not clearly visible. Please upload a clearer image."
            return txt, spec_pil

        # Stage 2: Breed Classification
        if species_name == "Cow":
            img_size = COW_IMG_SIZE
            interpreter = cow_interpreter
            labels = cow_labels
            emoji = "🐄"
        elif species_name == "Buffalo":
            img_size = BUFF_IMG_SIZE
            interpreter = buff_interpreter
            labels = buff_labels
            emoji = "🐃"
        else:
            txt = f"### {emoji} Species: **{species_name}** ({spec_conf*100:.2f}%)\n\n"
            txt += debug_info
            txt += "\n⚠️ Unknown species detected"
            return txt, spec_pil

        # Prepare and run breed prediction
        breed_pil = image_pil.resize((img_size, img_size))
        breed_tensor = np.expand_dims(np.array(breed_pil, dtype=np.float32), 0)
        breed_pred = run_tflite(interpreter, breed_tensor)

        # Get top 3 breeds
        top3_idx = breed_pred.argsort()[-3:][::-1]
        top3 = [(labels[i], float(breed_pred[i])) for i in top3_idx]

        # Calculate metrics
        ent = float(entropy(breed_pred))
        pur = purity(ent, top3[0][1], top3[1][1])

        # Format output with clear species info
        txt = f"### {emoji} Species: **{species_name}** — Confidence: `{spec_conf*100:.2f}%`\n\n"
        txt += debug_info  # Show debug info to verify correct species
        txt += "\n### 🏷️ Top 3 Breed Predictions\n"
        for i, (breed_name, prob) in enumerate(top3, 1):
            bar = "█" * int(prob * 20)
            txt += f"**{i}. {breed_name}** — `{prob*100:.2f}%` {bar}\n"

        txt += f"\n### 📊 Metrics\n**Entropy:** `{ent:.4f}` | **Status:** {pur}"

        return txt, breed_pil

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"### ❌ Error\n{str(e)}\n\n**Details:**\n```\n{error_details}\n```", image_pil

# ------------------------
# UPLOAD MODE
# ------------------------

def pipeline(image):
    """Process uploaded image"""
    if image is None:
        return "### Please upload an image", None

    try:
        text, processed_img = run_species_and_breed(image)
        return text, processed_img

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"### ❌ Error\n{str(e)}\n\n**Details:**\n```\n{error_details}\n```", None

# ------------------------
# GRADIO UI
# ------------------------

with gr.Blocks(title="Cattle Breed Classifier", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # 🐄🐃 Indian Cattle Breed AI Classifier
    ### 2-Stage Pipeline: Species Classification → Breed Identification
    """)

    gr.Markdown("Upload a clear photo of a cow or buffalo for breed identification")

    with gr.Row():
        with gr.Column():
            img = gr.Image(type="pil", label="Upload Cattle Image")
            analyze_btn = gr.Button("🔍 Analyze Breed", variant="primary", size="lg")
        with gr.Column():
            crop = gr.Image(label="Processed Image")

    breed = gr.Markdown(label="Classification Results")

    analyze_btn.click(pipeline, inputs=[img], outputs=[breed, crop])

    gr.Markdown("""
    ---
    ### 📋 Supported Breeds

    **Cow Breeds (17):** Alambadi, Gir, Hariana, Holstein Friesian, Kangayam, Kankrej, Khillari, Rathi, Red Kandhari, Red Sindhi, Sahiwal, Tharparkar, Bachaur, Bhelai, Gangatari, Gaolao, Kherigarh

    **Buffalo Breeds (6):** Jaffrabadi, Mehsana, Murrah, Neli Ravi, Banni, Pandharpuri

    ---
    ### 💡 Tips for Best Results
    - Use clear, well-lit photos
    - Capture the animal from the side or front
    - Ensure the animal is clearly visible
    - Avoid blurry or dark images

    ---
    ### 🔧 Troubleshooting
    If you see incorrect species predictions:
    1. Check the debug probabilities in the output
    2. If Buffalo shows >90% but it says "Cow", swap the species_labels order in the code
    3. Ensure your image is clear and well-lit
    4. Try different angles of the same animal
    """)

# Launch with public URL
print("\n🚀 Launching Gradio app...\n")
demo.launch(share=True, server_port=7860, debug=True)

✅ Setup complete!
Loading models...
✓ Species classifier loaded (input size: 256)
✓ Cow breed classifier loaded (input size: 256)
✓ Buffalo breed classifier loaded (input size: 256)

🎉 All models loaded successfully!



/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
/tmp/ipython-input-987404770.py:230: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Cattle Breed Classifier", theme=gr.themes.Soft()) as demo:



🚀 Launching Gradio app...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://970e231ae9d91afba3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",